In [1]:
'''
Potential evapotranspiration PM-FAO56 from R script
'''

import datetime
from datetime import datetime
import os
import glob
import math
import configparser
import numpy as np
import rasterio as rio
import xarray
import rioxarray
from pathlib import Path

In [2]:
config = configparser.ConfigParser()
config.read('et0_config.ini')

Z_U = float(config["et0_param"]["Z_U"]) # wind speed "sensor/model" height (m)
Z_Ta = float(config["et0_param"]["Z_Ta"]) # air temperature "sensor/model" height (m)
r_hc = float(config["et0_param"]["r_hc"]) # crop height (m)
r_leaf = float(config["et0_param"]["r_leaf"]) # leaf stomatal resistence (s.m-1)
r_LAI = float(config["et0_param"]["r_LAI"]) # LAI
r_albedo = float(config["et0_param"]["r_albedo"]) # Albedo
latdegr = float(config["et0_param"]["latdegr"]) # Latitude (decimal degree)
zo1 = float(config["et0_param"]["zo1"])
altezza = float(config["et0_param"]["altezza"]) # Altezza sul livell del mare del punto di campionamento

d = 2*r_hc/3
z0 = 0.05*r_hc

r_ra = ((math.log((Z_U-d)/(0.123*r_hc)))*(math.log((Z_Ta-d)/(0.0123*r_hc))))/0.1681    #  k^2 (v.Karman const)=0.1681

indir1 = Path("EODATA/")
indir2 = Path("METEODATA/")
outdir2 = Path("OUTPUT/")

inFile_T = glob.glob(str(indir2) + '/*_t2m.tif') # T media giornaliera da ERA5
inFile_Tmax = glob.glob(str(indir2) + '/*_tmax.tif') # T massima giornaliera da ERA5
inFile_Tmin = glob.glob(str(indir2) + '/*_tmin.tif') # T minima giornaliera da ERA5
inFile_Tdew = glob.glob(str(indir2) + '/*_d2m.tif') # Temperatura di rugiada da ERA5 (media giornaliera)
inFile_ssrd = glob.glob(str(indir2) + '/*_ssrd.tif') # Surface solar radiation downwards da ERA5. Cumulato, prendere ultimo valore orario
inFile_u = glob.glob(str(indir2) + '/*_u.tif') # Velocità vento da ERA5
inFile_sp = glob.glob(str(indir2) + '/*_sp.tif') # Surface pressure (Pa) da ERA5 (media giornaliera)
###
inFile_sisc = glob.glob(str(indir2) + '/*_SISC.tif') # Surface Indirect Shortwave Radiation W/m^2

card = len(inFile_T)

In [3]:
for j in range(card):
    Tavg = rioxarray.open_rasterio(inFile_T[j], masked=True)
    Tmax = rioxarray.open_rasterio(inFile_Tmax[j], masked=True)
    Tmin = rioxarray.open_rasterio(inFile_Tmin[j], masked=True)
    atmP = rioxarray.open_rasterio(inFile_sp[j], masked=True) #ERA5 in Pa da convertire in kPA
    U = rioxarray.open_rasterio(inFile_u[j], masked=True)
    SR = rioxarray.open_rasterio(inFile_ssrd[j], masked=True) # ATTENZIONE dati in J/m^2 devono essere convertiti in W/mq
    ###
    SISC = rioxarray.open_rasterio(inFile_sisc[j]) # Dati in W/m^2
    ###
    Tdew = rioxarray.open_rasterio(inFile_Tdew[j], masked=True)
    
    RH = 100*(np.exp((17.625*Tdew)/(243.04+Tdew))/np.exp((17.625*Tavg)/(243.04+Tavg))) # Relative Humidity (%)

    #Day of the Year
    file_name = (inFile_T[j])[10:20]
    date = file_name[:10]
    date_object = datetime.strptime(date, "%Y-%m-%d")
    doy = date_object.timetuple().tm_yday
    
    #AIR HUMIDITY
    #es: mean saturation vapour pressure  
    esmax = 0.6108*np.exp(17.27*Tmax/(Tmax+237.3))
    esmin = 0.6108*np.exp(17.27*Tmin/(Tmin+237.3))
    es = (esmax+esmin)/2
#     print(es[0], esmax[0], esmin[0])
#     es = 0.6108*np.exp(17.27*Tavg/(Tavg+237.3))


#     #DELTA: Slope of saturation vapour pression (KPa/°C)
    DELTA = (4098*es)/((Tavg+237.3)**2)

    #ea: actual vapour pressure derived from relative humidity data
#     ea = (es*RH/100)
    ea = 0.6108*np.exp(17.27*Tdew/(Tdew+237.3))

    #(es-ea): Vapour pressure deficit (KPa)
    vpd = (es-ea)
    
#     #ATMOSPHERIC PARAMETERS
    #LAMBDA = (2.501-0.002361*Tavg) #(MJ/Kg)- latent heat of vaporization = 2.45 (MJ/Kg)
    LAMBDA = 2.45
    cp = 1.013*1e-3   # air specific heat j/kg/°C
    atmkP = atmP/1000
    GAMMA = (0.622 * 1e-3)*atmkP
#     GAMMA = cp*atmkP/(0.622*LAMBDA) #Psychrometric constant (KPa/°C)
    rho = 3.486*atmkP/(Tavg+273)
    
    #RADIATION
    nSR = SR*(10**-6) # J/mq to MJ/mq*d
    
    distEaSun = 1+0.033*np.cos(2*np.pi*doy/365)     #inverse relative distance Earth-Sun
    soldecim = 0.409*np.sin((2*np.pi*doy/365)-1.39) #Solar decimal (rad)
    latitude = (np.pi/180)*latdegr               #Latitude (rad)
    sunsethangl = np.arccos(-np.tan(latitude)*np.tan(soldecim)) #Sunset hour angle (rad)
    Ra = (24 * 60 * 0.082 / np.pi) * distEaSun * (sunsethangl * np.sin(latitude) * np.sin(soldecim)
                                                  + np.cos(latitude) * np.cos(soldecim)
                                                  * np.sin(sunsethangl)) # extraterrestrial radiation for daily periods Ra (MJ/m^2/d)
    
    Rso=(0.75+(2*altezza)/100000)*Ra    # Clear Sky solar radiation (W/m^2)
    Rns = (1 - 0.23) * (nSR) # Net solar or net shortwave radiation (W/m^2)
#     Rnl = (0.0000000056745) * ((Tavg + 273.3) ** 4) * (0.34 - 0.14 * np.sqrt(ea)) * ((1.35 * (nSR) / Rso) - 0.35) # net longwave radiation (Rnl)(W.m-2)
    TmaxK4 = ((Tmax+273)**4)*4.903*1e-9
    TminK4 = ((Tmin+273)**4)*4.903*1e-9
#     Rnl = 2.539
    Rnl = ((TmaxK4+TminK4)/2)*(0.34 - 0.14 * np.sqrt(ea)) * (((1.35 * (nSR) / Rso)) - 0.35)
    Rn = (Rns - Rnl)

    # SOIL HEAT FLUX (G)
    G = 0.00
#   G = (0.2 * np.exp(-0.5 * r_LAI)) * (Rn) # (W/m^2)
    #r_rs = r_leaf / (0.5 * r_LAI)
    #flag3 = r_rs / r_ra
    #GAMMA1 = (1 + flag3 * U) * GAMMA
    #ETrad1 = (DELTA / (DELTA + GAMMA1)) * ((Rn - G) / LAMBDA) * 86400
    #ETaero1 = (86400 * rho * cp * vpd * U / r_ra / LAMBDA) / (DELTA + GAMMA1)
    ET1= ((0.408*Rn)*DELTA)/(DELTA+(GAMMA*(1+(0.34*U))))
    ET2=(900/(Tavg+273))*U*vpd*GAMMA/(DELTA+(GAMMA*(1+(0.34*U))))
    ET0 = ET1+ET2
#     ET0 = ETrad1 + ETaero1  # mm*d-1
#     ET0 = np.max(ET0, 0)
#     ET0 = ((0.408*DELTA*(Rn-G))+(LAMBDA*(900/(Tavg+273))*U*vpd))/(DELTA+LAMBDA*(1+(0.34*U)))
#     ET0 = ((0.408*DELTA*(Ra-G))+(LAMBDA*(900/Tavg+273)*U*vpd))/(DELTA+LAMBDA*(1+0.34*U))
    
    
    output_ET0 = str(outdir2) + '/' + inFile_T[j][10:20] + '_ET0'
    ET0.rio.to_raster(f"{output_ET0}" + ".tif", dtype=ET0.dtype, nodata=9999)
    
#     Calcolo Rn per input Aquacrop - FACOLTATIVO
#     output_ET1 = str(outdir2) + '/' + inFile_T[j][10:20] + '_ET1'
#     ET1.rio.to_raster(f"{output_ET1}" + ".tif", dtype=Rn.dtype, nodata=9999)
    
#     Calcolo Rn per input Aquacrop - FACOLTATIVO
#     output_ET2 = str(outdir2) + '/' + inFile_T[j][10:20] + '_ET2'
#     ET2.rio.to_raster(f"{output_ET2}" + ".tif", dtype=Rn.dtype, nodata=9999)    

#     Output DELTA facoltativi
#     output_DELTA = str(outdir2) + '/' + inFile_T[j][10:20] + '_DELTA'
#     DELTA.rio.to_raster(f"{output_DELTA}" + ".tif", dtype=Rn.dtype, nodata=9999)
    
 #     Output GAMMA facoltativi
#     output_GAMMA = str(outdir2) + '/' + inFile_T[j][10:20] + '_GAMMA'
#     GAMMA.rio.to_raster(f"{output_GAMMA}" + ".tif", dtype=Rn.dtype, nodata=9999)
    
#     Calcolo Rn per input Aquacrop - FACOLTATIVO
#     output_Rn = str(outdir2) + '/' + inFile_T[j][10:20] + '_Rn'
#     Rn.rio.to_raster(f"{output_Rn}" + ".tif", dtype=Rn.dtype, nodata=9999)
    
#     Calcolo Rnl per input Aquacrop - FACOLTATIVO
#     output_Rnl = str(outdir2) + '/' + inFile_T[j][10:20] + '_Rnl'
#     Rnl.rio.to_raster(f"{output_Rnl}" + ".tif", dtype=Rnl.dtype, nodata=9999)

#     Calcolo Rns per input Aquacrop - FACOLTATIVO
#     output_Rns = str(outdir2) + '/' + inFile_T[j][10:20] + '_Rns'
#     Rns.rio.to_raster(f"{output_Rns}" + ".tif", dtype=Rns.dtype, nodata=9999)
    
#     Calcolo nSR per input Aquacrop - FACOLTATIVO
#     output_nSR = str(outdir2) + '/' + inFile_T[j][10:20] + '_nSR'
#     nSR.rio.to_raster(f"{output_nSR}" + ".tif", dtype=Rns.dtype, nodata=9999)

#     print(ET0)
#     print(ET0.ndim)    

In [4]:
 
Tmax = 33.57 #rioxarray.open_rasterio(inFile_Tmax[j], masked=True)
Tmin = 20.30 #rioxarray.open_rasterio(inFile_Tmin[j], masked=True)
atmP = 99522 # in PA #rioxarray.open_rasterio(inFile_sp[j], masked=True) #ERA5 in Pa da convertire in kPA
U = 1.06  #rioxarray.open_rasterio(inFile_u[j], masked=True)
SR = 28401562 #rioxarray.open_rasterio(inFile_ssrd[j], masked=True) # ATTENZIONE dati in J/mq devono essere convertiti in W/mq
Tdew = 17.88 #rioxarray.open_rasterio(inFile_Tdew[j], masked=True)
Tavg = (Tmax+Tmin)/2 #rioxarray.open_rasterio(inFile_T[j], masked=True)
    
#     Tavg = (Tmax+Tmin)/2
    
#     Tavg = rio.open(inFile_T[j]).read()     #2m temperature daily average (24h) (C)
#     atmP = rio.open(inFile_sp[j]).read()    #Surface pressure (Pa)
#     U = rio.open(inFile_u[j]).read()     #wind speed 2 (m s-1)  Rescaled from 10 to 2m by using [eq. 47] Chapter 3 - FAO56 https://www.fao.org/3/x0490e/x0490e07.htm
#     SR = rio.open(inFile_ssrd[j]).read()  #Surface solar radiation downwards W m-2
#     Tdew = rio.open(inFile_Tdew[j]).read()  #2m dewpoint temperature daily average (24h) (C)
RH = 100*(np.exp((17.625*Tdew)/(243.04+Tdew))/np.exp((17.625*Tavg)/(243.04+Tavg))) # Relative Humidity (%)
 

#Day of the Year
# file_name = (inFile_T[j])[10:20]
# date = file_name[:10]
# date_object = datetime.strptime(date, "%Y-%m-%d")
# doy = date_object.timetuple().tm_yday
doy = 180

#AIR HUMIDITY
#es: mean saturation vapour pressure  
esmax = 0.6108*np.exp(17.27*Tmax/(Tmax+237.3))
esmin = 0.6108*np.exp(17.27*Tmin/(Tmin+237.3))
es = (esmax+esmin)/2
#     print(es[0], esmax[0], esmin[0])
#     es = 0.6108*np.exp(17.27*Tavg/(Tavg+237.3))


#     #DELTA: Slope of saturation vapour pression (KPa/°C)
DELTA = (4098*es)/((Tavg+237.3)**2)

    #ea: actual vapour pressure derived from relative humidity data
#     ea = (es*RH/100)
ea = 0.6108*np.exp(17.27*Tdew/(Tdew+237.3))

    #(es-ea): Vapour pressure deficit (KPa)
vpd = (es-ea)
    
#     #ATMOSPHERIC PARAMETERS
    #LAMBDA = (2.501-0.002361*Tavg) #(MJ/Kg)- latent heat of vaporization = 2.45 (MJ/Kg)
LAMBDA = 2.45
cp = 1.013*1e-3   # air specific heat j/kg/°C
atmkP = atmP/1000
# GAMMA = 0.0662
GAMMA = (0.622 * 1e-3)*atmkP
#     GAMMA = cp*atmkP/(0.622*LAMBDA) #Psychrometric constant (KPa/°C)
rho = 3.486*atmkP/(Tavg+273)
    
#RADIATION
nSR = SR*(10**-6) # J/mq to MJ/mq*d
    
distEaSun = 1+0.033*np.cos(2*np.pi*doy/365)     #inverse relative distance Earth-Sun
soldecim = 0.409*np.sin((2*np.pi*doy/365)-1.39) #Solar decimal (rad)
latitude = (np.pi/180)*latdegr               #Latitude (rad)
sunsethangl = np.arccos(-np.tan(latitude)*np.tan(soldecim)) #Sunset hour angle (rad)
Ra = (24 * 60 * 0.082 / np.pi) * distEaSun * (sunsethangl * np.sin(latitude) * np.sin(soldecim)
                                                + np.cos(latitude) * np.cos(soldecim)
                                               * np.sin(sunsethangl)) # extraterrestrial radiation for daily periods Ra (MJ/m^2/d)
    
Rso=(0.75+(2*altezza)/100000)*Ra    # Clear Sky solar radiation (W/m^2)
Rns = (1 - 0.23) * (nSR) # Net solar or net shortwave radiation (W/m^2)
#     Rnl = (0.0000000056745) * ((Tavg + 273.3) ** 4) * (0.34 - 0.14 * np.sqrt(ea)) * ((1.35 * (nSR) / Rso) - 0.35) # net longwave radiation (Rnl)(W.m-2)
TmaxK4 = ((Tmax+273)**4)*4.903*1e-9
TminK4 = ((Tmin+273)**4)*4.903*1e-9
# Rnl = 2.539
Rnl = ((TmaxK4+TminK4)/2)*(0.34 - 0.14 * np.sqrt(ea)) * (((1.35 * (nSR) / Rso)) - 0.35)
Rn = (Rns - Rnl)
  
# SOIL HEAT FLUX (G)
G = 0.00
#   G = (0.2 * np.exp(-0.5 * r_LAI)) * (Rn) # (W/m^2)
    #r_rs = r_leaf / (0.5 * r_LAI)
    #flag3 = r_rs / r_ra
    #GAMMA1 = (1 + flag3 * U) * GAMMA
    #ETrad1 = (DELTA / (DELTA + GAMMA1)) * ((Rn - G) / LAMBDA) * 86400
    #ETaero1 = (86400 * rho * cp * vpd * U / r_ra / LAMBDA) / (DELTA + GAMMA1)
ET1= ((0.408*Rn)*DELTA)/(DELTA+(GAMMA*(1+(0.34*U))))
ET2=(900/(Tavg+273))*U*vpd*GAMMA/(DELTA+(GAMMA*(1+(0.34*U))))
ET0 = ET1+ET2
   
    
print(f'DELTA : {(DELTA)}')
print(f'GAMMA : {(GAMMA)}')
print(f'Rns : {(Rns)}')
print(f'Rnl : {(Rnl)}')
print(f'Rn : {(Rn)}')
print(f'ET1 : {(ET1)}')
print(f'ET2 : {(ET2)}')
print(f'ET0 : {(ET0)}')
print(f'Giorno in DOY           : {int(doy)}')
print(f'Vento  a 2 m di altezza :   {round(U,2)}   m\s')
print(f'T. max a 2 m di altezza :  {round(Tmax,2)}  °C')
print(f'T. min a 2 m di altezza :  {round(Tmin,2)}  °C')
print(f'T. dew a 2 m di altezza :  {round(Tdew,2)}  °C')
print(f'******************ET0***:   {ET0}\n\n')


DELTA : 0.22231053217211869
GAMMA : 0.061902684000000006
Rns : 21.86920274
Rnl : 4.851351908653343
Rn : 17.017850831346657
ET1 : 5.035724086888194
ET2 : 1.1171476531987783
ET0 : 6.152871740086972
Giorno in DOY           : 180
Vento  a 2 m di altezza :   1.06   m\s
T. max a 2 m di altezza :  33.57  °C
T. min a 2 m di altezza :  20.3  °C
T. dew a 2 m di altezza :  17.88  °C
******************ET0***:   6.152871740086972


